# IFT870 - Forage de données - TP2

Membres de l'équipe :
- Joë Larue
- Aurélien Vauthier

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# setup notebook parameters
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# extract data
package = pd.read_csv("Package2.csv", delimiter=';', encoding="latin1")
product = pd.read_csv("Product2.csv", delimiter=';', encoding="latin1")

# 1 Auscultation

Une première observation des données grâce à microsoft Excel, ainsi que des descriptions en ligne sur https://www.fda.gov/ nous a permis d'avoir un premier aperçu du contenu de nos tables et leurs significations.

In [3]:
print(f"Affichons un aperçu des premières observations de product (taille : {product.shape} :")
product.head()

Affichons un aperçu des premières observations de product (taille : (93238, 20) :


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


In [2]:
print(f"Affichons un aperçu des premières observations de package (taille : {package.shape} :")
package.head()

Affichons un aperçu des premières observations de package (taille : (173887, 8) :


,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


Notre première observation est qu'il existe plus de Package que de Product, ce qui nous laisse penser qu'un produit sera probablement présent dans plusieurs packages. 

## 1.1 Correctitude des données

In [13]:
product.dtypes

PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGRED_UNIT                   object
PHARM_CLASSES                        object
DEASCHEDULE                          object
NDC_EXCLUDE_FLAG                     object
LISTING_RECORD_CERTIFIED_THROUGH    float64
dtype: object

In [17]:
package.dtypes

PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object

Le type objet signifie qu'au sein d'une même colonne, il y a un mélange de plusieurs types.  
Certains attributs représentent des dates et n'ont pas le format idéal comme:
    
    STARTMARKETINGDATE      
    ENDMARKETINGDATE    
    LISTING_RECORD_CERTIFIED_THROUGH    

## 1.2 Complétude

In [28]:
product.isna().sum()

PRODUCTID                            1560
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
ROUTENAME                            1932
STARTMARKETINGDATE                      0
ENDMARKETINGDATE                    88915
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                             0
SUBSTANCENAME                        2309
ACTIVE_NUMERATOR_STRENGTH            2309
ACTIVE_INGRED_UNIT                   2309
PHARM_CLASSES                       50984
DEASCHEDULE                         88815
NDC_EXCLUDE_FLAG                        0
LISTING_RECORD_CERTIFIED_THROUGH     4325
dtype: int64

In [27]:
package.isna().sum()

PRODUCTID                  0
PRODUCTNDC              1500
NDCPACKAGECODE          2346
PACKAGEDESCRIPTION         0
STARTMARKETINGDATE         0
ENDMARKETINGDATE      167431
NDC_EXCLUDE_FLAG           0
SAMPLE_PACKAGE             0
dtype: int64

De nombreuses valeurs sont manquantes, cependant il est fort probable que des valeurs étaient facilement récupérables en croisant les 2 tables comme pour le PRODUCTID par exemple ou alors en se fiant à la description.

## 1.3 Cohérence

Le PRODUCTNDC est présent dans la construction de plusieurs autres attributs :  
    PRODUCTID              
    NDCPACKAGECODE          
    
Le MARKETINGCATEGORYNAME est présent dans la construction de APPLICATIONNUMBER.

le PACKAGEDESCRIPTION a un lien avec :  
    SUBSTANCENAME         
    ACTIVE_NUMERATOR_STRENGTH    
    ACTIVE_INGRED_UNIT      
    
LISTING_RECORD_CERTIFIED_THROUGH a un lien avec NDC_EXCLUDE_FLAG.

L'utilisation d'un script pour vérifier la cohérence entre ces lignes peut-être intéressantes, mais il faudrait d'abord réduire le nombre de valeurs manquantes.  
Pour le moment, les données semblent cohérentes sur ce point à vue d'oeil.

## 1.4 Données à jour

LISTING_RECORD_CERTIFIED_THROUGH représente la date de fin de validité des informations sur un médicament. Beaucoup de valeurs sont manquantes, cependant la présence de 'N' et non pas 'E' dans la colonne de NDC_EXCLUDE_FLAG nous indique que les informations devraient être à jour.

On peut faire 2 hypothèses, les lignes où LISTING_RECORD_CERTIFIED_THROUGH est manquant ne sont pas à jour, et donc la ligne NDC_EXCLUDE_FLAG est fausse et on ne traite pas ces lignes. Ou alors on se fie à la ligne NDC_EXCLUDE_FLAG est-on les traites quand même. 

Nous allons suivre la 2e hypothèse et continuer à utiliser les données en question.

## 1.5 Données vraisemblables et interprétables

Nous allons passer au cas par cas les attributs et identifier le maximum de choses à dire à leur sujet.

### Attributs présent dans les 2 tables:

***PRODUCTID***  
Comme identifié au-dessus, il y a des valeurs manquantes dans la tables product uniquement.  
Il y a aussi des éléments en double dans la table des packages, ce qui a du sens à priori.  
Le productid est la fusion entre product NDC (connue) et SPL document ID (inconnue).  
Ce code ne possède pas d'informations pertinentes en dehors de son utilité pour fusionner les 2 tables.

***PRODUCTNDC***  
Cet attribut est la concaténation du code de l'étiqueteur/se et du code du produit. Ces 2 éléments vont de pair pour identifier un médicament.  
Les différents formats sont 4-4, 5-3, ou 5-4 (ex:12345-1234).  
Des valeurs sont manquantes dans la table package, il se peut qu'une récupération puisse être faite grâce au PRODUCTID.  
En parcourant le fichier Product, nous avons trouvé des données aberrantes comme des dates par exemple.  

***STARTMARKETINGDATE***  
En parcourant la colonne, nous avons identifié la présence de bruits.  

Par exemple, la date 29971224 doit probablement correspondre à 19971225 avec une faute de frappe.  

Pour ce qui est de 19000101, cela doit probablement être dû au fait que la donnée est inconnue.  

In [47]:
print("Valeur Max dans Product: " + str(product["STARTMARKETINGDATE"].max()))
print("Valeur Min dans Product: " + str(product["STARTMARKETINGDATE"].min()))
print("Valeur Max dans Package: " + str(package["STARTMARKETINGDATE"].max()))
print("Valeur Min dans Package: " + str(package["STARTMARKETINGDATE"].min()))

Valeur Max dans Product: 20200214
Valeur Min dans Product: 19000101
Valeur Max dans Package: 29971224
Valeur Min dans Package: 19000101


***ENDMARKETINGDATE***  
Cela représente une date, au format AAAAMMJJ.  
En parcourant la colonne, nous avons identifié la présence de bruits ou de données aberrantes.  
Cependant, le très grand nombre de valeurs manquantes dans cette colonne réduit considérablement l'importance que nous allons lui attribuer. 

In [48]:
print("Valeur Max dans Product: " + str(product["ENDMARKETINGDATE"].max()))
print("Valeur Min dans Product: " + str(product["ENDMARKETINGDATE"].min()))
print("Valeur Max dans Package: " + str(package["ENDMARKETINGDATE"].max()))
print("Valeur Min dans Package: " + str(package["ENDMARKETINGDATE"].min()))

Valeur Max dans Product: 30310209.0
Valeur Min dans Product: 20200215.0
Valeur Max dans Package: 20390831.0
Valeur Min dans Package: 20200215.0


***NDC_EXCLUDE_FLAG***  
D'après la description des 2 tables, il semblerait qu'il soit normal que l'on n'ait que des 'N' dans cette colonne. Les objets ayant des valeurs différentes ont été mis dans des tables à part que nous ne possédons pas.

In [50]:
print("Nombre de valeur unique dans package: " + str(package["NDC_EXCLUDE_FLAG"].nunique()))
print("Nombre de valeur unique dans Product: " + str(product["NDC_EXCLUDE_FLAG"].nunique()))

Nombre de valeur unique dans package: 1
Nombre de valeur unique dans Product: 1


### Attributs présent uniquement dans la table Product:

***PRODUCTTYPENAME***  
Pas de données manquantes, et les valeurs semblent correctes. 

***PROPRIETARYNAME***  
Cet attribut décrit le nom du médicament.  
Beaucoup de valeurs différentes sont présentes mais aussi des valeurs qui se ressemblent.  
Il est difficile d'identifier comment nettoyer les données autrement qu'au cas par cas. 

In [57]:
product["PROPRIETARYNAME"][93068:93072]

93068             ARNICA 30C
93069                 ARNICA
93070              Arnica 6c
93071    Arsenicum Album 30c
Name: PROPRIETARYNAME, dtype: object

***PROPRIETARYNAMESUFFIX***  
Cet attribut est optionnel, ce qui explique le très grand nombre de cases vide. Le manque de standard de rédaction réduit l'intérêt à porter sur cet attribut. 

***NONPROPRIETARYNAME***  
Les valeurs écrites dans cet attribut peuvent être sous forme de liste. Cependant les listes peuvent utiliser des séparateurs tels que ',' ou 'and' et certains mots sont présents en majuscules / minuscules.

Quelques valeurs sont manquantes.

In [68]:
product["NONPROPRIETARYNAME"][543]

'Butalbital, Aspirin, Caffeine, and Codeine Phosphate'

***DOSAGEFORMNAME***  
Cet attribut décrit l'apparence du médicament, les valeurs semblent correctes et il n'y a pas de valeur est manquante.

***ROUTENAME***  
Cet attribut définit les voies d'administration du médicament par le patient. Plusieurs voies peuvent être utilisées, et ces valeurs sont séparées par des ';'.
Des valeurs sont manquantes.

***MARKETINGCATEGORYNAME***  
Cet attribut semble correct car il n'y a pas de valeur manquante.  
Cet attribut possède un lien étroit avec l'attribut suivant.

***APPLICATIONNUMBER***  
Cet attribut dépend du précédent. On y retrouve les codes NDA/ANDA/BLA, sinon "part X" pour les catégories OTC ou l'absence de valeur pour les "UNAPPROVED".

***LABELERNAME***  
Certaines valeurs sont aberrantes, d'autres valides mais similaires.

In [72]:
product["LABELERNAME"][1]

'10'

In [73]:
product["LABELERNAME"][25788:25790]

25788                      Meijer
25789    Meijer Distribution Inc.
Name: LABELERNAME, dtype: object

***SUBSTANCENAME***  
Cet attribut représente la liste de substances dans le médicament.
Beaucoup de valeurs sont manquantes.

***ACTIVE_NUMERATOR_STRENGTH***  
Cet attribut représente la quantité de substances décrites dans SUBSTANCENAME dans le médicament.
La valeur manquante quand SUBSTANCENAME est manquant.

***ACTIVE_INGRED_UNIT***  
Cet attribut représente l'unité des substances décrites dans SUBSTANCENAME dans le médicament.
La valeur manquante quand SUBSTANCENAME est manquant.

***PHARM_CLASSES***  
Cet attribut représente les classes pharmaceutiques auxquelles appartient le médicament. Il peut y en avoir plusieurs par ligne, séparée par des ','.

Beaucoup de données sont manquantes.

***DEASCHEDULE***  
Cet attribut peut prendre les valeurs de {CI, CII, CIII, CIV, CV}.
Beaucoup de valeurs sont manquantes, et il n'y a aucun CI.
Notre hypothèse est que les CI on était supprimé, il faut donc remplacer les cases vides par CI. 

***LISTING_RECORD_CERTIFIED_THROUGH***  
Cet attribut représente la fin de validité de l'information contenue dans une ligne. Certaines données sont manquantes, les autres semblent correctes. 

In [74]:
print("Valeur Max pour LISTING_RECORD_CERTIFIED_THROUGH: " + str(product["LISTING_RECORD_CERTIFIED_THROUGH"].max()))
print("Valeur Min pour LISTING_RECORD_CERTIFIED_THROUGH : " + str(product["LISTING_RECORD_CERTIFIED_THROUGH"].min()))

Valeur Max pour LISTING_RECORD_CERTIFIED_THROUGH: 20211231.0
Valeur Min pour LISTING_RECORD_CERTIFIED_THROUGH : 20201231.0


### Attributs présent uniquement dans la table Package:

***NDCPACKAGECODE***  
Cet attribut prend la valeur du PRODUCTNDC + un code correspondant au package, sous le format 4-4-2, 5-3-2, ou 5-4-1.
Des valeurs sont manquantes et d'autres aberrantes, comme la présence de dates.

In [79]:
package["NDCPACKAGECODE"][41901]

'20210901'

***PACKAGEDESCRIPTION***  
Ceci est une zone de description contenant un duplicata du NDCPACKAGECODE et décrit la quantité de médicaments dans chaque package et sous-package.
Aucune donnée n'est manquante.

***SAMPLE_PACKAGE***  
Cet attribut présente un 'Y' quand un package est à distribuer en tant qu'échantillon.
Pas de valeur manquante, et les données sont correctes. 

## 1.6 Résumé

La qualité des données est 'à revoir'.  
Les types présents dans les colonnes sont à uniformiser.  
Beaucoup de cases sont vides mais il y en a un bon nombre qui est une valeur facilement retrouvable.  
Certains attributs ont des valeurs qui manquent de cohérence, notamment au niveau des dates.  
Pour certaines données, on ne sait pas si elles sont à jour, notre hypothèse est qu'elles le sont.  
Un mélange de bruits et de données aberrantes est présent dans l'ensemble des 2 tables. Dans la plupart des cas, c'est facilement identifiable. 

# Autre

In [31]:
### Colonnes en double
- package contient plus de lignes que product
- PRODUCTID : données manquantes (chez product uniquement) & fusion entre product NDC et SPL document ID & duplicatas (ex 30901-30904)
- PRODUCTNDC : concaténation du code de l'étiqueteuse et du produit & données manquantes chez package (possibilité de récupération via PRODUCTID) & données abérantes chez product (dates ex: 320 (product) et 36655 (package))
- STARTMARKETINGDATE : date au format (AAAAMMJJ) & valeur bruité dans package (ex 33 avec 2997 au lieu de 1997)
- ENDMARKETINGDATE : date au format (AAAAMMJJ), valeurs abérantes dans product (ex 29505 avec année 3031)
- NDC_EXCLUDE_FLAG : uniquement la valeur 'N' car les autres produit (marqué 'Y', 'E' ou 'I', sont dans un autre fichier)

#### Colonnes de product seulement
- PRODUCTTYPENAME : pas de données manquantes
- PROPRIETARYNAME : valeurs abérantes (similaires ex 93068-93072, infos supplémentaires ex 430, erreur ex 7414)
- PROPRIETARYNAMESUFFIX : valeurs manquantes & optionnel & manque de standard
- NONPROPRIETARYNAME : valeurs abérantes (similaires : ',' et 'and', majuscules / minuscules)
- DOSAGEFORMNAME : liste d'attribut représentant l'apparence
- ROUTENAME : valeurs manquantes & liste d'attribut représentant les voies d'administrations
- MARKETINGCATEGORYNAME : en lien avec APPLICATIONNUMBER & valeurs similaires (ex UNAPPROVED DRUG OTHER, UNAPPROVED MDECIAL GAS...)
- APPLICATIONNUMBER : soit numéro NDA/ANDA/BLA ou "part X" pour les catégories OTC ou manquante pour les "UNAPPROVED" 
- LABELERNAME : valeurs abérantes ("10") & valeurs similaires (ex 25788-25790)
- SUBSTANCENAME : listes de substances & valeurs manquantes
- ACTIVE_NUMERATOR_STRENGTH (aka STRENGTHNUMBER) : quantité pour chaque substance (cf SUBSTANCENAME) & valeurs manquantes (lorsque substances manquantes)
- ACTIVE_INGRED_UNIT (aka STRENGTHUNIT) : liste des unité de mesure pour chaque quantité (cf ACTIVE_NUMERATOR_STRENGTH) & valeurs manquantes (lorsque substances manquantes)
- PHARM_CLASSES : valeurs manquantes & liste de classe
- DEASCHEDULE : valeurs manquantes & énumeration {CI, CII, CIII, CIV, CV} & CI non présente (peut-être null = CI ?) 
- LISTING_RECORD_CERTIFIED_THROUGH : valeurs manquantes & date d'expiration du statut à jour de la ligne (au format AAAAMMJJ) & peut-être redondant avec NDC_EXCLUDE_FLAG ?

Les colonnes SUBSTANCENAME, ACTIVE_NUMERATOR_STRENGTH et ACTIVE_INGRED_UNIT forme un trio de listes qui ont la même taille pour chaque médicaments (ou sont toutes les trois manquantes)

#### Colonnes de package seulement
- NDCPACKAGECODE : valeur de PRODUCTNDC + un code correspondant au package & valeurs abérantes (date ex 41903)
- PACKAGEDESCRIPTION : zone de description contenant un duplicata du NDCPACKAGECODE & décrit la quantité de médicament dans chaque package et sous-package
- SAMPLE_PACKAGE : 'Y' quand un package est à distribuer comme en échantillon.

Il y a des ProductID manquant dans le fichier `product`

93084

Les colonnes de type object signifit qu'il y a plusieurs types qui se mélange. 
Par défault, nous utiliserons le type string, cependant il existe des cas particuliés pour:
    
STARTMARKETINGDATE : int64 Les dates étants écrits sous le format AAAAMMJJ, le type int peut simplifier la visualisation.

ENDMARKETINGDATE : int64 Cet attribut est aussi une date.

ACTIVE_NUMERATOR_STRENGTH : float64 Seule des chiffres, et certain à virgules, sont utilisé.

LISTING_RECORD_CERTIFIED_THROUGH : int64 Cet attribut est aussi une date.

Les types vont être uniformiser en String sauf pour :

STARTMARKETINGDATE : int64 Cet attribut est une date.

ENDMARKETINGDATE : int64 Cet attribut est aussi une date.

***La proposition de changement de type de certains attributs dans les 2 tables sont provisoires.***  
Certains attributs comportent des listes d'éléments, il se peut que le changement en vecteur boolean puisse être intéressant.

product["PRODUCTID"].astype("string")
product["PRODUCTNDC"].astype("string")
product["PRODUCTTYPENAME"].astype("string")
product["PROPRIETARYNAME"].astype("string")
product["PROPRIETARYNAMESUFFIX"].astype("string")
product["NONPROPRIETARYNAME"].astype("string")
product["DOSAGEFORMNAME"].astype("string")
product["ROUTENAME"].astype("string")
product["STARTMARKETINGDATE"].astype("int64")
product["ENDMARKETINGDATE"].astype("int64")
product["MARKETINGCATEGORYNAME"].astype("string")
product["APPLICATIONNUMBER"].astype("string")
product["LABELERNAME"].astype("string")
product["SUBSTANCENAME"].astype("string")
product["ACTIVE_NUMERATOR_STRENGTH"].astype("float64")
product["ACTIVE_INGRED_UNIT"].astype("string")
product["PHARM_CLASSES"].astype("string")
product["DEASCHEDULE"].astype("string")
product["NDC_EXCLUDE_FLAG"].astype("string")
product["LISTING_RECORD_CERTIFIED_THROUGH"].astype("int64")

package["PRODUCTID"].astype("string")
package["PRODUCTNDC"].astype("string")
package["NDCPACKAGECODE"].astype("string")
package["STARTMARKETINGDATE"].astype("int64")
package["ENDMARKETINGDATE"].astype("int64")
package["NDC_EXCLUDE_FLAG"].astype("string")
package["SAMPLE_PACKAGE"].astype("string")

